# Social Computing/Social Gaming - Summer 2020

# Exercise Sheet 3: Collaborative Filtering with Steam Games

In this exercise, we will build a collaborative filtering recommender system using data we gather from Steam. We will use your friends list to get information about owned games for each ID, and the time each game was played.

Usually, collaborative filtering is based on some sort of rating to determine the similarity between users. However, for games, the enjoyment and a rating do not always match. Additionally, only about 10% of players actually rate the games they play, which would make for a very incomplete dataset. Therefore, the playtime will be used instead of a rating system. This has the added benefit that playtime is usually the most authentic metric of enjoyment, as players are very unlikely to spend much time on a game they don't enjoy.

## Task 3.1: Obtaining the data


**1.** Your first task is to gather the data needed to create the recommender system. Create a data structure that holds the needed information for each player and game.  
**Note:** You cannot obtain a list from your profile with the Steam API unless your profile is set to public. 

If you do not have a Steam profile, you can use the default values. 
However, we encourage you to use your own profile. 

**Hint**: To obtain the games a user owns, use this: `games = data['response']['games']`. This returns a list of games, including the playtime (in minutes) which can be retrieved like this: `playtime = game['playtime_forever']` , where game refers to an item from the list of games. 

In [1]:
#Use this if you want to work with the default IDs
import requests
import urllib
import pandas as pd
import json
from urllib.request import Request, urlopen
from pandas.io.json import json_normalize
from requests.exceptions import HTTPError


# You can replace these values with your own ID and API key
key = "CB35B8F8DCE9135DDAA3B0328FCE0103"
my_id = "76561198329838242" 
url = "http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key="+key+"&steamids="+ my_id
r = requests.get(url)
data = r.json()

# Get friendslist
request = Request("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key="+key+"&steamid="+ my_id +"&relationship=friend")
response = urlopen(request)
elevations = response.read()
data = json.loads(elevations)
friendslist = data['friendslist']
friends = friendslist['friends']

friendids =[]
tempIDs = []
for friend in friends:
    friendids.append(friend['steamid'])
    
print(len(friendids))
#get friends of friends:
x = 0
while x < len(friendids):
    friendID = friendids[x]
    request = Request("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key="+key+"&steamid="+friendID+"&relationship=friend")
    try:
        response = urlopen(request)    
    except urllib.error.HTTPError  as e:
        print('401')
    elevations = response.read()
    try:
        data = json.loads(elevations)
    except json.JSONDecodeError:
        print('couldnt decode')
    friendslist = data['friendslist']
    friends = friendslist['friends']

    friendidsNew =[]
    for friend in friends:
        friendidsNew.append(friend['steamid'])
        
    tempIDs+=friendidsNew
    x+=1

friendids += tempIDs
friendids = list(dict.fromkeys(friendids))
friendids = list(set(friendids))
print(len(friendids))


64
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
401
couldnt decode
6657


In [2]:
# Trim the list of IDs to reasonable values:
if len(friendids)>250:
    friendids = friendids[:250]    
print(f'len_friendids: {len(friendids)}')

users_gamedicts = {} # The dictionary containing all information for every ID
gamedict = {} # A dict containing information for one player

# Get owned games of friendslist:
request = Request("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="+key+"&steamid="+my_id+"&include_appinfo=1&format=json")

# TODO:
# Open the URL and read the json response and retrieve the games of your friends and their playtime
# Save the games into a dictionary with key=name and values=playtime
# Hint 1: You can obtain the games a user owns with data['response']['games']
# Hint 2: You can retrieve their playtime with game['playtime_forever']
try:
    response = urlopen(request)
except urllib.error.HTTPError as e:
        print(f'401: {e}')

elevations = response.read()

try:
    data = json.loads(elevations)
    gamedict = {game['name']: game['playtime_forever'] for game in data['response']['games'] if game['playtime_forever'] != 0}
except json.JSONDecodeError:
        print('couldnt decode')
        
# Add the dictionary to the users_gamedict       
users_gamedicts[int(my_id)]=gamedict

# Do the same for the friends of your friends
for friendID in friendids:
    # TODO
    request = Request("http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=" + key + "&steamid=" + friendID + "&include_appinfo=1&format=json")
    try:
        response = urlopen(request)
    except urllib.error.HTTPError as e:
        print(f'401: {e}')

    elevations = response.read()
    
    try:
        data = json.loads(elevations)
        if data['response'].get('games'):
            gamedict = {game['name']: game['playtime_forever'] for game in data['response']['games'] if game['playtime_forever'] != 0}
            if len(gamedict) > 0:
                users_gamedicts[int(friendID)] = gamedict
            
    except json.JSONDecodeError:
        print('couldnt decode')

print(f'len_users_gamedicts: {len(users_gamedicts)}')

len_friendids: 250
len_users_gamedicts: 30


<!-- # import pickle

# with open('friendids.pkl', 'wb') as file:
#     pickle.dump(friendids, file)

# with open('users_gamedicts.pkl', 'wb') as file:
#     pickle.dump(users_gamedicts, file) -->

<!-- # my_id = "76561198329838242"
# friendids = []
# users_gamedicts = {}

# with open('friendids.pkl', 'rb') as file:
#     friendids = pickle.load(file)

# with open('users_gamedicts_73.pkl', 'rb') as file:
#     users_gamedicts = pickle.load(file)


# print(len(friendids))
# print(len(users_gamedicts)) -->

## Task 3.2: Association rule mining

Before we start with the "real" recommender system, let us take a look at a more general form of recommending items using association rules.

The concept of association rule mining is rather simple: Looking at an itemset, one tries to find dependencies between items that could "belong together". A common example would be buying food at the store: If, for example, meat and salt are bought together often, but meat without salt not that often, it is assumed that there is a connection between those two. For games, if it was found that most of the users who own the demo version of a game also own the full version of that game, it would be a reasonable assumption that these users liked the demo and therefore bought the full version.


Let us first cover the mathematical basis for association rules. The most important metrics used are **support**,  **confidence** and **lift**. *Support is defined as the amount of times an item occurs in the itemset divided by the total number of items in the set*; *Confidence is defined as the support of a list of items [x,y,...] divided by the support of x*. *Lift is a measure describing the correlation between items*. Written down mathematically:

$$supp(x)= \frac{len(x)}{len(n)}$$

$$conf(x=>y) = \frac{supp(x,y)}{supp(x)}$$

$$lift(x=>y) = \frac{P(x \cap y)}{P(x) * P(y)}$$



It is important to note that *support refers to an item or a list of items, while confidence refers to a rule*. Also note that *a lift of 1 means that x and y occur independently of each other, while a lift greater 1 means a positive correlation*.


**1.** Your task here is to first convert the dictionary you created into a list of lists as this is the input required for the algorithm to work. Then, print out the most frequent items using the `min_support` attribute. Finally, print out the association rules and play around with the threshold value to get a reasonable amount of rules. 


**2.** Discuss your results and try to answer the following questions: What kind of recommendations can be made? What does a confidence of 1.0 mean and is it meaningful for recommending games? Can you spot a correlation between the games with the highest support and the rules with the highest confidence? How does this affect the lift?  
**Hint:** Play around with the threshold values until you get a reasonable amount (4-30) rows as output.

In [3]:
gamesofallusers = [list(gamedict.keys()) for fid, gamedict in users_gamedicts.items()]

# TODO: Convert the gamedict to a list of lists

# Remove common Steam entries that are not games:
for game in gamesofallusers:
    if 'Dota 2 Test' in game:
        game.remove('Dota 2 Test')
    if 'True Sight' in game:
        game.remove('True Sight')
    if 'True Sight: Episode 1' in game:
        game.remove('True Sight: Episode 1')
    if 'True Sight: Episode 2' in game:
        game.remove('True Sight: Episode 2')
    if 'True Sight: Episode 3' in game:
        game.remove('True Sight: Episode 3')
    if 'True Sight: The Kiev Major Grand Finals' in game:
        game.remove('True Sight: The Kiev Major Grand Finals')
    if 'True Sight: The International 2017' in game:
        game.remove('True Sight: The International 2017')
    if 'True Sight: The International 2018 Finals' in game:
        game.remove('True Sight: The International 2018 Finals') 

In [4]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
# TODO: Tinker around with the values
te_ary = te.fit(gamesofallusers).transform(gamesofallusers)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.40, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, ignore_index=True)

,support,itemsets
0,0.833333,(Counter-Strike: Global Offensive)
1,0.500000,(PAYDAY 2)
2,0.466667,(Path of Exile)
3,0.466667,"(PAYDAY 2, Counter-Strike: Global Offensive)"
4,0.433333,(Left 4 Dead 2)
5,0.433333,(PLAYERUNKNOWN'S BATTLEGROUNDS)
6,0.433333,(Warframe)
7,0.433333,"(PLAYERUNKNOWN'S BATTLEGROUNDS, Counter-Strike..."
8,0.400000,(Garry's Mod)
9,0.400000,(The Elder Scrolls V: Skyrim)


In [5]:
from mlxtend.frequent_patterns import association_rules

# TODO: Play around with the treshold value
df_conf = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.40)
df_conf.sort_values(by='confidence', ascending=False, ignore_index=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLAYERUNKNOWN'S BATTLEGROUNDS),(Counter-Strike: Global Offensive),0.433333,0.833333,0.433333,1.000000,1.200000,0.072222,inf
1,(The Elder Scrolls V: Skyrim),(Counter-Strike: Global Offensive),0.400000,0.833333,0.400000,1.000000,1.200000,0.066667,inf
2,(PAYDAY 2),(Counter-Strike: Global Offensive),0.500000,0.833333,0.466667,0.933333,1.120000,0.050000,2.500000
3,(Left 4 Dead 2),(Counter-Strike: Global Offensive),0.433333,0.833333,0.400000,0.923077,1.107692,0.038889,2.166667
4,(Warframe),(Counter-Strike: Global Offensive),0.433333,0.833333,0.400000,0.923077,1.107692,0.038889,2.166667
5,(Path of Exile),(Counter-Strike: Global Offensive),0.466667,0.833333,0.400000,0.857143,1.028571,0.011111,1.166667
6,(Counter-Strike: Global Offensive),(PAYDAY 2),0.833333,0.500000,0.466667,0.560000,1.120000,0.050000,1.136364
7,(Counter-Strike: Global Offensive),(PLAYERUNKNOWN'S BATTLEGROUNDS),0.833333,0.433333,0.433333,0.520000,1.200000,0.072222,1.180556
8,(Counter-Strike: Global Offensive),(Left 4 Dead 2),0.833333,0.433333,0.400000,0.480000,1.107692,0.038889,1.089744
9,(Counter-Strike: Global Offensive),(Path of Exile),0.833333,0.466667,0.400000,0.480000,1.028571,0.011111,1.025641


**TODO: Write your observations here:**

**Q**: *What kind of recommendations can be made?*\
**A**: Recommendations of popular games and recommendations of games that are frequently bought together can be made based on games owned by the target user. From a rule, one can also recommend consequent games to a user given the antecedent games currently owned by that user. However, unlike collaborative filtering, these kinds of recommendations do not require analyzing user preferences.


**Q**: *What does a confidence of $1.0$ mean and is it meaningful for recommending games?*\
**A**: A rule with a confidence of $1.0$ means every single user that has the antecedent game, has also bought the corresponding consequent game. This can be seen in the formula for calculating confidence: for a confidence of 1.0, it should be the case that $supp(X,Y) = supp(X)$ given that we know the target user has already ordered itemset X. This means itemset X appears together with itemset Y in every single transaction with itemset X in it. Therefore, every user that has X also owns Y. Therefore, for such rules with confidence of 1.0, if we know a user already bought games X, it is not meaningful at all to recommend games Y to him because we know for sure he also has games Y already.

**Q**: *Can you spot a correlation between the games with the highest support and the rules with the highest confidence?*\
**A**: When a game with the highest support is the consequent item of a rule, that rule always has a very high confidence value. The confidence for those rules is high because the antecedent itemset is such a frequent one and would be almost present in every transaction. For example, the table of association rules above shows that rules with game `Counter-Strike: Global Offensive` (highest support itemset) as their consequent item always have the highest confidence. Those are the first 6 rules in the table. Therfore, this is the pitfall of inferring rules by confidence measure.

**Q**: *How does this affect the lift?*\
**A**: Games with highest support as the consequent of a rules do not affect the lift. As we can see from the rule table 
above, lift values of the first 6 rules with consequent having the highest support are more uniformly distributed compared to the remaining rules. All lift values among all 22 rules only slightly vary inside the interval $(1.0, 1.2]$ considering the possible value range for a lift is $[0, \infty)$, which indicates that the causal relations between the antecent and the consequent in these rules are almost independent ones. Especially for the first 6 rules in the table, they are not such strong causal relations as their respective confidence values suggest. However, all of the rules show positive correlations between the repsective antecedent and consequent, although just slightly positive. So we can say that lift is a measure more resilient itemsets with high support than confidence.

## Task 3.3: The Recommender System: Similarity Score


Finally, it is time to build the recommender system. 

**1.** The first thing to do is to implement a similarity score that will be used to predict a user's playtime of an unowned game. We implement a similarity score between two users by taking the relative distance between two players. We use the following formula:

$$d(u, v) = \sum_{i~\in~\textrm{common_games}} \frac{|r_{u,i} - r_{v,i}|}{r_{v,i}}$$  

Where $u$ and $v$ are users and $r_{u,i}$ is the playtime of user $u$ for game $i$. 

You can then return the similarity with  
$$ w_{u,v} = \frac{1}{1 + d(u, v)} $$

**Note:** If no common games exist return 0.

In [6]:
# Here we will calculate the similarity score between two friends based on their common games:
def calculate_similarity(user1ID, user2ID):
    # TODO:
    games1 = users_gamedicts[user1ID]
    games2 = users_gamedicts[user2ID]

    common_games = games1.keys() & games2.keys()

    if not common_games:
        return 0.0

    distance = 0.0
    for game in common_games:
        playtime1 = games1[game]
        playtime2 = games2[game]
        distance += abs(playtime1 - playtime2) / float(playtime2)

    return 1.0 / (1.0 + distance)

## Task 3.4: Recommender System: Predict ratings

With the similarity score calculated, we can now predict a user's playtime for games they don't own.   
**1.** First, we create a set of all games, but we delete all games that are owned by less than 3 players. The reason is simple: If only 1 or 2 players own a game, it is impossible to derive a meaningful prediction since there is not enough data. 

The predicted playtime for a game works analogous to the predicted rating of a movie/item in a conventional collaborative filtering recommender system:

$$r_{u,i} = \frac{\sum_{v \in N_i(u)} w_{u,v}r_{v,i}}{\sum_{v \in N_i(u)} w_{u,v}}$$

where 
- $r_{u,i}$ is the estimated recommendation of item $i$ for target user $u$. 
- $N_i(u)$ is the set of similar users to target user $u$ for the designated item $i$. 
- $w_{u,v}$ is the similarity score between users $u$ and $v$ (used as a weighting factor).  

**Note:** In our case, we use playtime as a recommendation measure and the set $N_i(u)$ consists of user $u$ friends list and friends of friends list. In our scenario, we do not need the index $i$ as our friends list does not change between games.

In [7]:
# List of all games that are owned by at least 1 person:
allGames = []
for user in gamesofallusers:
    for game in user:
        allGames.append(game)
        
# TODO : Create a list of games owned by at least 3 people
from collections import defaultdict

games_ownercounts = defaultdict(int)
for game in allGames:
    games_ownercounts[game] += 1
allGames = [game for game, num_owner in games_ownercounts.items() if num_owner >= 3]

_my_id = int(my_id)

# Find out which games you do not own out of all games because we are only interested in recommendations for games that we do not own
def difference(allGames, usersgames): 
    # TODO:
    return list(set.difference(set(allGames), set(usersgames)))

    
# Predict ratings based on the formula above for each unowned game
def predict_ratings():
    # TODO:
    """
    Hint: Iterate over all unowned games and for each game calculate a rating based 
    on your friends playtime and similarity score
    """
    games_users_mapping = defaultdict(list)
    for u_id, games in users_gamedicts.items():
        for game in games:
            games_users_mapping[game].append(u_id)

    pred_playtimes = {}
    
    unowned_games = difference(allGames, users_gamedicts[_my_id])
    for game in unowned_games:
        friend_ids = games_users_mapping[game]
        weighted_playtime_sum = 0.0  # numerator
        w_sum = 0.0  # denominator
        for friend_id in friend_ids:
            w = calculate_similarity(_my_id, friend_id)
            w_sum += w

            v_playtime = users_gamedicts[friend_id][game]
            weighted_playtime_sum += w * v_playtime

        pred_playtimes[game] = weighted_playtime_sum / w_sum if w_sum != 0.0 else 0.0

    return pred_playtimes

## Task 3.5: Recommender System: Discussion

**1.** Sort the predicted ratings by estimated playtime (highest first) and print out the top 5 predictions for you (or the default user if you are using the default ID). 

**2.** Discuss the difference in recommendations between the collaborative filtering approach and the association rule approach. Would you consider one more accurate than the other? Why/why not?

In [8]:
# TODO 1: 
pred_ratings = predict_ratings()
df_pred_ratings = pd.DataFrame([{'recommended_games': game, 'predicted_playtime': playtime} 
                                for game, playtime in pred_ratings.items()])
df_pred_ratings.sort_values(by='predicted_playtime', ascending=False, ignore_index=True, inplace=True)
df_pred_ratings.iloc[:5,:]

,recommended_games,predicted_playtime
0,ARK: Survival Evolved,292865.673400
1,EVE Online,107485.993609
2,Counter-Strike,63060.563144
3,Counter-Strike: Global Offensive,52247.209813
4,MONSTER HUNTER: WORLD,47011.943092


**TODO 2: Write your observations here:**


**Q**:*Discuss the difference in recommendations between the collaborative filtering approach and the association rule approach?*\
**A**: Collaborative Filtering (CF) is more user-centric than Association Rule (AR) approach. CF takes user personal preferences into account and measures the similarity between the target user and users with same preferences e.g. his/her friends, while AR derives recommendations by considering just transaction history of the whole population/user-base, and then the consequent itemset in each rule will be taken as recommendations to the target user based on the antecedent itemset/games currently owned by that user.

**Q**: *Would you consider one more accurate than the other? Why/why not?*\
**A**: CF is to some extend more accurate than AR, since it uses more data related to the target user such as personal preferences, friend list, ordered game list, playtime history, etc., while AR only uses ordered game list as antecedents to derive rules.\
As a rough caparison for our case, with those 12 rules from the rule table of task 3.2,  we are only able to recommend the target user (myself) game `Counter-Strike: Global Offensive` from rule `Path of Exile` $\rightarrow$ `Counter-Strike: Global Offensive` by the AR approach although this rule is pretty weak with lift value very closed to $1.0$, indicating these two games are almost indepedent of one another. There's only one game with large uncertainty can be taken for recommendation by AR because among games owned by the target user, which are `Path of Exile`, `Europa Universalis IV`, `Titan Quest Anniversary Edition`, `Black Desert Online`, `Crusader Kings II`, only `Path of Exile` shows up in the "antecedents" column, even though the `min_support` threshold is set pretty low to $0.40$ and the `min_threshold` is also low to $0.40$.
On the other hand, we can recommend the 5 games in the table above derived by CF (where game `Counter-Strike: Global Offensive` will also be recommended by CF) with high confidence that the target user will also like those games based on the large predicted playtime inferred from his friends, and therefore, increase the probability that the user will buy those games next.

In [9]:
# Games owned by the target user (myself)
print(list(users_gamedicts[int(my_id)].keys()))

['Path of Exile', 'Europa Universalis IV', 'Titan Quest Anniversary Edition', 'Black Desert Online', 'Crusader Kings II']


### References

https://michael.hahsler.net/research/association_rules/measures.html#confidence

https://towardsdatascience.com/association-rules-2-aa9a77241654

https://en.wikipedia.org/wiki/Association_rule_learning